In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## About this Dataset

This data has been collected from two solar power plants in India over a 34-day period. It represents both power generation and weather data from two distinct plants. The power generation datasets are collected at the inverter level, with each inverter connected to multiple lines of solar panels. Weather data is gathered using sensors positioned at the plant level, utilizing a single array of sensors strategically placed within each plant.

After a preliminary examination of both plant datasets, it is evident that the second plant's data exhibits a significant number of missing values, with up to 30% of the power generation data being incomplete. As a result, I have chosen to focus exclusively on the data from plant 1 for this study, which has a more manageable 5% rate of missing values.

## Load and Clean data

In [2]:
power_data = pd.read_csv("./Plant_1_Generation_Data.csv")
weather_data = pd.read_csv("./Plant_1_Weather_Sensor_Data.csv")

### 1- Power generation data

In [3]:
power_data.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
1,15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
2,15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
3,15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
4,15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0


In [4]:
power_data.shape

(68778, 7)

In [5]:
power_data.dtypes

DATE_TIME       object
PLANT_ID         int64
SOURCE_KEY      object
DC_POWER       float64
AC_POWER       float64
DAILY_YIELD    float64
TOTAL_YIELD    float64
dtype: object

In [6]:
# the feature "PLANT_ID" seems to have the same value 4136001 and thus should be safely dropped
print("feature 'PLANT_ID' count:", len(power_data["PLANT_ID"].unique()))
power_data = power_data.drop(["PLANT_ID"], axis=1)

feature 'PLANT_ID' count: 1


In [7]:
# there are 22 inverters in this plant
inverters = power_data["SOURCE_KEY"].unique()
print(f"Plant inverter count: {len(inverters)}")
print(f"Plant inverters:\n {inverters}")

Plant inverter count: 22
Plant inverters:
 ['1BY6WEcLGh8j5v7' '1IF53ai7Xc0U56Y' '3PZuoBAID5Wc2HD' '7JYdWkrLSPkdwr4'
 'McdE0feGgRqW7Ca' 'VHMLBKoKgIrUVDU' 'WRmjgnKYAwPKWDb' 'ZnxXDlPa8U1GXgE'
 'ZoEaEvLYb1n2sOq' 'adLQvlD726eNBSB' 'bvBOhCH3iADSZry' 'iCRJl6heRkivqQ3'
 'ih0vzX44oOqAx2f' 'pkci93gMrogZuBj' 'rGa61gmuvPhdLxV' 'sjndEbLyjtCKgGv'
 'uHbuxQJl8lW7ozc' 'wCURE6d3bPkepu2' 'z9Y9gH1T5YWrNuG' 'zBIq5rxdHJRwDNY'
 'zVJPv84UY57bAof' 'YxYtjZvoooNbGkE']


In [8]:
# change the inverters names for simplicity
power_data["INVERTER"] = power_data["SOURCE_KEY"].map({v: f'INV_{k}' for k, v in enumerate(inverters)})
power_data = power_data.drop(["SOURCE_KEY"], axis=1)

In [9]:
# verify missing data for each inverter data
expected_sample_per_inverter = 34*24*4 # (34days with a 15min sampling period) 
(power_data["INVERTER"].value_counts() / expected_sample_per_inverter) * 100 

INVERTER
INV_10    96.660539
INV_0     96.629902
INV_3     95.986520
INV_5     95.986520
INV_7     95.894608
INV_12    95.894608
INV_18    95.772059
INV_17    95.772059
INV_16    95.741422
INV_13    95.741422
INV_11    95.741422
INV_14    95.710784
INV_15    95.710784
INV_4     95.710784
INV_20    95.710784
INV_8     95.680147
INV_1     95.557598
INV_9     95.557598
INV_19    95.557598
INV_6     95.526961
INV_2     95.526961
INV_21    95.098039
Name: count, dtype: float64

It appears that all inverters have missing values, accounting for approximately 5% of the total data.

In [10]:
# Check if all datetimes are present using frequency
power_data["DATE_TIME"] = pd.to_datetime(power_data["DATE_TIME"], format='%d-%m-%Y %H:%M')
print(f"Data datetime frequency : {power_data['DATE_TIME'].unique().freq}")

Data datetime frequency : None


The absence of a returned frequency (None) indicates the presence of missing datetimes. To identify these gaps, we can compare the datetimes in the original dataset with a 15-min date range created for the same time period.

In [11]:
# construct a 34 days with 15min intervals from data start-end dates 
datetimes = power_data["DATE_TIME"]
datetime_fullrange = pd.date_range(start=min(datetimes), end=max(datetimes), freq='15min')
print(f"Missing datetimes: {len(datetime_fullrange.difference(datetimes.unique()))}")

Missing datetimes: 106


### Filling out missing datetimes

It's important to observe that the missing values identified above concerns to the entire dataset. Additionally, when we examined the missing values for individual inverters, we notice that some of them have a higher number of missing samples, indicating a greater number of missing dates.

To address these gaps and complete the missing datetimes for each inverter, we need to separately reindex the datetimes for each inverter and subsequently merge them back together.

In [12]:
fullrange = pd.Series(datetime_fullrange, name='DATE_TIME')

data = []
for inverter in power_data["INVERTER"].unique():
    df = power_data[power_data.INVERTER == inverter]
    df = df.merge(fullrange, on='DATE_TIME', how='right')
    df['INVERTER'] = inverter
    data.append(df)
power_df = pd.concat(data)

In [13]:
# Check new data for missing values
power_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71808 entries, 0 to 3263
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATE_TIME    71808 non-null  datetime64[ns]
 1   DC_POWER     68778 non-null  float64       
 2   AC_POWER     68778 non-null  float64       
 3   DAILY_YIELD  68778 non-null  float64       
 4   TOTAL_YIELD  68778 non-null  float64       
 5   INVERTER     71808 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
# check for null values
power_df.isna().sum()

DATE_TIME         0
DC_POWER       3030
AC_POWER       3030
DAILY_YIELD    3030
TOTAL_YIELD    3030
INVERTER          0
dtype: int64

It is expected that the data now contains NaN values, which were introduced when we added all the missing timestamps for each inverter. To use this data, we must first address these missing values using one of the common filling methods: the use of previous/next item or interpolation.

Since our data is not truly continuous due to solar panels not generating power and yield during nighttime, we will consider a forward-fill method in our study. However, it's important to note that this approach may introduce some inaccuracies, particularly when dealing with missing values at the transition between day and nighttime.

In [15]:
power_df = power_df.fillna(method='ffill')
power_df.isna().sum()

DATE_TIME      0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
TOTAL_YIELD    0
INVERTER       0
dtype: int64

### 2- Weather data

In [16]:
weather_data.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,4135001,HmiyD2TTLFNqkNe,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,4135001,HmiyD2TTLFNqkNe,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,4135001,HmiyD2TTLFNqkNe,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,4135001,HmiyD2TTLFNqkNe,24.621525,22.165423,0.0


In [17]:
print("feature 'PLANT_ID' count:", len(weather_data["PLANT_ID"].unique()))
print("feature 'SOURCE_KEY' count:", len(weather_data["SOURCE_KEY"].unique()))

feature 'PLANT_ID' count: 1
feature 'SOURCE_KEY' count: 1


Earlier, we mentioned that we can omit the 'PLANT_ID' feature since we're working with a single plant. Additionally, we can now exclude the inverter columns ('SOURCE_KEY' feature) from the data, this is because the weather data is collected at the location of one inverter, and we assume it to be consistent across all inverters, accurately representing the entire plant.

In [18]:
weather_data = weather_data.drop(["PLANT_ID", "SOURCE_KEY"], axis=1)

In [19]:
# check if there are missing values
weather_data.isna().sum()

DATE_TIME              0
AMBIENT_TEMPERATURE    0
MODULE_TEMPERATURE     0
IRRADIATION            0
dtype: int64

In [20]:
# check if there are missing datetimes
weather_data["DATE_TIME"] = pd.to_datetime(weather_data["DATE_TIME"], format='%Y-%m-%d %H:%M:%S')
print(f"Data datetime frequency : {weather_data['DATE_TIME'].unique().freq}")

Data datetime frequency : None


In [21]:
datetimes = weather_data["DATE_TIME"]
datetime_fullrange = pd.date_range(start=min(datetimes), end=max(datetimes), freq='15min')
print(f"Missing datetimes: {len(datetime_fullrange.difference(datetimes.unique()))}")

Missing datetimes: 82


### Filling out missing values

No datetime frequency indicates that the weather data has also some missing timestamps, which we must add to be able to combine with the power generation data.

After adding the missing date-times, we need to fill in all the null values introduced in the other features : 'AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION'. Since all of these features are continuous in nature, we will use interpolation.

In [22]:
cols = weather_data.columns
weather_data = weather_data.set_index('DATE_TIME').reindex(datetime_fullrange).reset_index()
weather_data.columns = cols

In [24]:
# Use interpolation to fill missing values
weather_data['AMBIENT_TEMPERATURE'].interpolate(method='linear', inplace=True)
weather_data['MODULE_TEMPERATURE'].interpolate(method='linear', inplace=True)
weather_data['IRRADIATION'].interpolate(method='linear', inplace=True)

In [25]:
weather_data

,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,25.184316,22.857507,0.0
1,2020-05-15 00:15:00,25.084589,22.761668,0.0
2,2020-05-15 00:30:00,24.935753,22.592306,0.0
3,2020-05-15 00:45:00,24.846130,22.360852,0.0
4,2020-05-15 01:00:00,24.621525,22.165423,0.0
...,...,...,...,...
3259,2020-06-17 22:45:00,22.150570,21.480377,0.0
3260,2020-06-17 23:00:00,22.129816,21.389024,0.0
3261,2020-06-17 23:15:00,22.008275,20.709211,0.0
3262,2020-06-17 23:30:00,21.969495,20.734963,0.0


In [26]:
# augment weather data to account for multiple inverters
weather_df = pd.concat([weather_data for i in range(len(inverters))])

In [27]:
# combine the power generation and weather dataframes
data = pd.merge(power_df, weather_data, on="DATE_TIME")

In [29]:
data

,DATE_TIME,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,INVERTER,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,0.0,0.0,0.000,6259559.0,INV_0,25.184316,22.857507,0.0
1,2020-05-15 00:00:00,0.0,0.0,0.000,6183645.0,INV_1,25.184316,22.857507,0.0
2,2020-05-15 00:00:00,0.0,0.0,0.000,6987759.0,INV_2,25.184316,22.857507,0.0
3,2020-05-15 00:00:00,0.0,0.0,0.000,7602960.0,INV_3,25.184316,22.857507,0.0
4,2020-05-15 00:00:00,0.0,0.0,0.000,7158964.0,INV_4,25.184316,22.857507,0.0
...,...,...,...,...,...,...,...,...,...
71803,2020-06-17 23:45:00,0.0,0.0,5147.625,7028601.0,INV_17,21.909288,20.427972,0.0
71804,2020-06-17 23:45:00,0.0,0.0,5819.000,7251204.0,INV_18,21.909288,20.427972,0.0
71805,2020-06-17 23:45:00,0.0,0.0,5817.000,6583369.0,INV_19,21.909288,20.427972,0.0
71806,2020-06-17 23:45:00,0.0,0.0,5910.000,7363272.0,INV_20,21.909288,20.427972,0.0


## Exploratory data analysis